# Evaluation

Evaluation and benchmarking play a pivotal role in the development of LLM Applications. For optimizing the performance of applications such as RAG (Retrieval Augmented Generation), a robust measurement mechanism is indispensable.

LlamaIndex offers vital modules tailored to assess the quality of generated outputs. Additionally, it incorporates specialized modules designed specifically to evaluate content retrieval quality. LlamaIndex categorizes its evaluation into two primary types:

*   **Response Evaluation**
*   **Retrieval Evaluation**

[Documentation
](https://gpt-index.readthedocs.io/en/latest/core_modules/supporting_modules/evaluation/root.html)

# Response Evaluation

Evaluating results from LLMs is distinct from traditional machine learning's straightforward outcomes. LlamaIndex employs evaluation modules, using a benchmark LLM like GPT-4, to gauge answer accuracy. Notably, these modules often blend query, context, and response, minimizing the need for ground-truth labels.

The evaluation modules manifest in the following categories:

*   **Faithfulness:** Assesses whether the response remains true to the retrieved contexts, ensuring there's no distortion or "hallucination."
*   **Context Relevancy:** Evaluates the relevance of both the retrieved context and the generated answer to the initial query.
*   **Correctness:** Determines if the generated answer aligns with the reference answer based on the query (this does require labels).

Furthermore, LlamaIndex has the capability to autonomously generate questions from your data, paving the way for an evaluation pipeline to assess the RAG application.

<b> Evaluation of RAG can be costly GPT-4 is being used. Please keep track of the cost. You can try to run on lesser data to reduce cost.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

# Set up the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Set logger level to INFO

# Clear out any existing handlers
logger.handlers = []

# Set up the StreamHandler to output to sys.stdout (Colab's output)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # Set handler level to INFO

# Add the handler to the logger
logger.addHandler(handler)

In [4]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [5]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Response,
)

In [6]:
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    RetrieverEvaluator,
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset
)

In [7]:
from llama_index.llms.openai import OpenAI

In [8]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

In [9]:
import os

In [10]:
from dotenv import load_dotenv, find_dotenv
# load_dotenv('D:/.env')
# OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [11]:
load_dotenv('D:/Training/FAA-Training/Beyond-the-Prompt-Practical-RAG-for-Real-World-AI/RAG-systems-using-LlamaIndex/RAG-System-Using-LamaIndex/.env')

True

### Download Data

In [12]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/9607a05a923ddf07deee86a56d386b42943ce381/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

The syntax of the command is incorrect.
'data/paul_graham/paul_graham_essay.txt': No such file or directory


### Load Data

In [13]:
reader = SimpleDirectoryReader("./data/paul_graham/")
documents = reader.load_data()

### Generate Questions

In [14]:
data_generator = RagDatasetGenerator.from_documents(documents, llm=OpenAI(temperature=0, model="gpt-4o-mini"),
                                                   num_questions_per_chunk=2)

In [15]:
eval_dataset = data_generator.generate_dataset_from_nodes()

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.co

In [16]:
eval_dataset.examples[0].query

"Describe the author's early experiences with programming on the IBM 1401. What challenges did he face, and how did these experiences shape his understanding of programming at that time?"

In [17]:
eval_dataset.examples[0].reference_answer

'The author\'s early experiences with programming on the IBM 1401 were marked by curiosity and confusion. He began programming at the age of 13 or 14, using an early version of Fortran on a machine that was part of his school district\'s data processing resources. The IBM 1401 was a complex system for him, resembling a "mini Bond villain\'s lair" with its array of alien-looking machines. \n\nOne of the main challenges he faced was the limitation of input methods; the only way to input data into the programs was through punched cards, and he did not have any data stored in that format. This lack of input options severely restricted what he could accomplish with the machine. He also struggled with the mathematical knowledge required to create interesting programs, which left him feeling puzzled about how to effectively use the computer. \n\nHis most memorable experience was realizing that programs could run indefinitely without terminating, which was a significant moment for him, highlig

In [18]:
eval_questions = [example.query for example in eval_dataset.examples]
eval_answers = [example.reference_answer for example in eval_dataset.examples]

In [22]:
len(eval_questions)

46

In [21]:
eval_questions[0]

"Describe the author's early experiences with programming on the IBM 1401. What challenges did he face, and how did these experiences shape his understanding of programming at that time?"

In [19]:
eval_answers[0]

'The author\'s early experiences with programming on the IBM 1401 were marked by curiosity and confusion. He began programming at the age of 13 or 14, using an early version of Fortran on a machine that was part of his school district\'s data processing resources. The IBM 1401 was a complex system for him, resembling a "mini Bond villain\'s lair" with its array of alien-looking machines. \n\nOne of the main challenges he faced was the limitation of input methods; the only way to input data into the programs was through punched cards, and he did not have any data stored in that format. This lack of input options severely restricted what he could accomplish with the machine. He also struggled with the mathematical knowledge required to create interesting programs, which left him feeling puzzled about how to effectively use the computer. \n\nHis most memorable experience was realizing that programs could run indefinitely without terminating, which was a significant moment for him, highlig

To be consistent we will fix evaluation query

In [20]:
eval_query = eval_questions[0]

<b> Check https://openai.com/pricing to select the less costlier variant of an LLM.<b>

In [23]:
# Fix gpt-4o-mini LLM for generating response
gpt4o_mini = OpenAI(temperature=0, model="gpt-4o-mini")

# Fix GPT-4o LLM for evaluation
gpt4 = OpenAI(temperature=0, model="gpt-4o")

In [24]:
# create vector index
vector_index = VectorStoreIndex.from_documents(
    documents, llm=OpenAI(temperature=0, model="gpt-4o")
)

# Query engine to generate response
query_engine = vector_index.as_query_engine()

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [25]:
retriever = vector_index.as_retriever(similarity_top_k=3)
nodes = retriever.retrieve(eval_query)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [26]:
from IPython.display import display, HTML
display(HTML(f'<p style="font-size:14px">{nodes[1].get_text()}</p>'))

## Context Relevency Evaluation

Measures if the response + source nodes match the query.

In [27]:
# Create RelevancyEvaluator using GPT-4 LLM
relevancy_evaluator = RelevancyEvaluator(llm=gpt4)

In [28]:
# Generate response
response_vector = query_engine.query(eval_query)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [29]:
response_vector.response

"The author's early experiences with programming on the IBM 1401 involved using an early version of Fortran where programs had to be typed on punch cards. He faced challenges due to the limited input options, as the only form of input was data stored on punched cards. This limitation meant he struggled to find meaningful tasks to perform with the computer. These experiences likely shaped his understanding of programming at that time by highlighting the importance of having accessible and interactive computing systems, as opposed to the batch processing method he encountered with the IBM 1401."

In [30]:
print(response_vector.get_formatted_sources())

> Source (Doc id: f99f1820-f485-416f-b3d6-29c1c5e95b70): What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...

> Source (Doc id: dc6c5486-005b-4d03-bb0b-39b0f81a4346): It felt like I was doing life right. I remember that because I was slightly dismayed at how novel...


In [31]:
# Evaluation
eval_result = relevancy_evaluator.evaluate_response(query=eval_query, response=response_vector)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [32]:
eval_result.query

"Describe the author's early experiences with programming on the IBM 1401. What challenges did he face, and how did these experiences shape his understanding of programming at that time?"

In [33]:
eval_result.response

"The author's early experiences with programming on the IBM 1401 involved using an early version of Fortran where programs had to be typed on punch cards. He faced challenges due to the limited input options, as the only form of input was data stored on punched cards. This limitation meant he struggled to find meaningful tasks to perform with the computer. These experiences likely shaped his understanding of programming at that time by highlighting the importance of having accessible and interactive computing systems, as opposed to the batch processing method he encountered with the IBM 1401."

In [34]:
eval_result.passing

True

Relevancy evaluation with multiple source nodes.

In [35]:
# Create Query Engine with similarity_top_k=3
query_engine = vector_index.as_query_engine(similarity_top_k=3)

# Create response
response_vector = query_engine.query(eval_query)

# Evaluate with each source node
eval_source_result_full = [
    relevancy_evaluator.evaluate(
        query=eval_query,
        response=response_vector.response,
        contexts=[source_node.get_content()],
    )
    for source_node in response_vector.source_nodes
]

# Evaluation result
eval_source_result = [
    "Pass" if result.passing else "Fail" for result in eval_source_result_full
]

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Retrying request to /chat/completions in 0.406157 seconds
Retrying request to /chat/completions in 0.406157 seconds
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Retrying request to /chat/completions in 0.416058 seconds
Retrying request to /chat/completions in 0.416058 seconds
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Retrying request to /chat/completions in 0.396580 seconds
Retrying request to /chat/completions in 0.396580 seconds
HTTP Request: POST https

In [36]:
eval_source_result

['Pass', 'Fail', 'Fail']

## Faithfullness Evaluator

 Measures if the response from a query engine matches any source nodes. This is useful for measuring if the response was hallucinated.

In [37]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=gpt4)

In [38]:
eval_result = faithfulness_evaluator.evaluate_response(response=response_vector)

Retrying request to /chat/completions in 0.431573 seconds
Retrying request to /chat/completions in 0.431573 seconds
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [40]:
eval_result.feedback

'YES'

In [39]:
eval_result.passing

True

In [41]:
eval_result.score

1.0

## Correctness Evaluator

Evaluates the relevance and correctness of a generated answer against a reference answer.

In [42]:
correctness_evaluator = CorrectnessEvaluator(llm=gpt4)

In [43]:
eval_reference_answer = eval_answers[0]

correctness_result = correctness_evaluator.evaluate(
    query=eval_query,
    response=response_vector.response,
    reference=eval_reference_answer,
)

Retrying request to /chat/completions in 0.376393 seconds
Retrying request to /chat/completions in 0.376393 seconds
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [44]:
correctness_result.score

4.0

In [45]:
correctness_result.passing

True

In [46]:
correctness_result.feedback

"The generated answer is relevant and mostly correct, capturing the essence of the author's early experiences with the IBM 1401, including the use of punch cards and the challenges faced. However, it lacks some specific details present in the reference answer, such as the author's age, the description of the IBM 1401, and the impact of these experiences on his later embrace of microcomputers."

## BatchEvalRunner - Run Evaluations in batch manner.

In [47]:
from llama_index.core.evaluation import BatchEvalRunner

runner = BatchEvalRunner(
    {
     "faithfulness": faithfulness_evaluator,
     "relevancy": relevancy_evaluator,
     "correctness": correctness_evaluator
     },
    workers=8,
)

eval_results = await runner.aevaluate_queries(
    query_engine, queries=eval_questions, reference = eval_answers
)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.

In [48]:
def get_eval_results(key, eval_results):
    results = eval_results[key]
    correct = 0
    for result in results:
        if result.passing:
            correct += 1
    score = correct / len(results)
    print(f"{key} Score: {score}")
    return score

In [49]:
_ = get_eval_results("faithfulness", eval_results)

faithfulness Score: 0.9565217391304348


In [50]:
_ = get_eval_results("relevancy", eval_results)

relevancy Score: 0.9130434782608695


In [51]:
_ = get_eval_results("correctness", eval_results)

correctness Score: 0.8913043478260869


## Benchmark using [LlamaDatasets](https://llamahub.ai/?tab=llama_datasets).

It's a 3 step process:

1. Download dataset
2. Build your RAG Pipeline
3. Evaluate using RagEvaluatorPack.

In [53]:
!llamaindex-cli download-llamapack RagEvaluatorPack --download-dir ./rag_evaluator_pack

Processing d:\training\faa-training\beyond-the-prompt-practical-rag-for-real-world-ai\rag-systems-using-llamaindex\rag-system-using-lamaindex\module4-evaluation-of-rag-systems\rag_evaluator_pack
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for llama-index-packs-rag-evaluator: filename=llama_index_packs_rag_evaluator-0.4.0-py3-none-any.whl size=4950 sha256=d38851c41376c4455a264bca29c4db29e7bbd2faccbe9506f1f38319d0d676d3
  Stored in directory: C:\Users\venkatasp\AppData\Local\Temp\pip-ephem-wheel-cache-25_m068c\wheels\2a\78\25\fe677ad6fa541058c524dae96438c08439e6c76c5a7e8a5fd5
Successfully built llama-index-packs-rag-evaluator
Successfully downloaded RagEvaluatorPack to ./rag_evaluator_pack



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
from llama_index.core.llama_dataset import LabelledRagDataset
from llama_index.packs.rag_evaluator import RagEvaluatorPack
from llama_index.core.llama_pack import download_llama_pack
from llama_index.core import VectorStoreIndex

Download the required files from the below link and move them to the folders mentioned below in the code

https://github.com/run-llama/llama-datasets/tree/main/llama_datasets/paul_graham_essay

In [55]:
rag_dataset = LabelledRagDataset.from_json("./data/rag_dataset.json")
documents = SimpleDirectoryReader(input_dir="./data/source_files").load_data()

In [56]:
rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,What are the key issues preventing the wide ad...,[From Bitcoin to Solana – Innovating Blockchai...,The key issues preventing the wide adoption of...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
1,How does blockchain technology provide data se...,[From Bitcoin to Solana – Innovating Blockchai...,Blockchain technology provides data security a...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
2,What is the projected growth rate of the block...,[2 \n \nchain market size is expected to grow ...,The projected growth rate of the blockchain ma...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
3,What are some of the challenges facing enterpr...,[2 \n \nchain market size is expected to grow ...,Some of the challenges facing enterprise adopt...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
4,What are the key issues that need to be addres...,[3 \n \nScalability. Scalability is the seco...,The key issues that need to be addressed in or...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
5,How can blockchains enhance security and integ...,[3 \n \nScalability. Scalability is the seco...,Blockchains can enhance security and integrity...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
6,How does Solana's blockchain technology addres...,[4 \n \ntionary nature of blockchain technolog...,Solana's blockchain technology addresses criti...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
7,Explain the main data structure of a blockchai...,[4 \n \ntionary nature of blockchain technolog...,The main data structure of a blockchain is a c...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
8,Explain the structure of the main body of each...,[5 \n \nTX Counter\nTX TX TXprevious block \nh...,The main body of each block in the blockchain ...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
9,Discuss the role and significance of smart con...,[5 \n \nTX Counter\nTX TX TXprevious block \nh...,Smart contracts play a crucial role in a block...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)


In [57]:
rag_dataset.examples[0] #query, reference_answer

LabelledRagDataExample(query='What are the key issues preventing the wide adoption of blockchain technology in enterprise applications, and how has Solana addressed these issues?', query_by=CreatedBy(model_name='gpt-3.5-turbo', type=<CreatedByType.AI: 'ai'>), reference_contexts=['From Bitcoin to Solana – Innovating Blockchain towards \nEnterprise Applications  \nXiangyu Li, Xinyu Wang , Tingli Kong , Junhao Zheng and Min Luo  \nGeorgia Institute of Technology, Atlanta, GA 30332, USA  \nmluo60@gatech.edu  \nAbstract. This survey presents a comprehe nsive study of recent advances in \nblockchain technologies ， focusing on how issues that affecting the enterprise \nadoption were progressively addressed from the original Bitcoin system to \nEthereum, to Solana etc. Key issues preventing the wide adoption ar e scalability \nand performance, while r ecent advances in Solana has clearly demonstrated that \nit is possible to significantly improve on those issues by innovating on data \nstructu

In [58]:
# build basic RAG system
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [63]:
rag_evaluator_pack = RagEvaluatorPack(rag_dataset=rag_dataset, query_engine=query_engine, judge_llm=gpt4)

In [64]:
# evaluate using the RagEvaluatorPack
benchmark_df = await rag_evaluator_pack.arun(
    batch_size=20,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)

Batch processing of predictions:   0%|          | 0/20 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.

Batch processing of predictions:   5%|▌         | 1/20 [00:02<00:49,  2.58s/it]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batch processing of predictions:  10%|█         | 2/20 [00:02<00:21,  1.17s/it]

...

Batch processing of evaluations: 14it [00:45,  3.84s/it]                          

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.

Batch processing of evaluations: 15it [00:50,  4.01s/it]

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.o

Batch processing of evaluations:  94%|█████████▍| 12/12.8 [00:54<00:03,  4.52s/it]


In [65]:
benchmark_df

rag,base_rag
metrics,
mean_correctness_score,4.181034
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.942123


# Retrieval Evaluation

Evaluates the quality of any Retriever module defined in LlamaIndex.

To assess the quality of a Retriever module in LlamaIndex, we use metrics like hit-rate and MRR. These compare retrieved results to ground-truth context for any question. For simpler evaluation dataset creation, we utilize synthetic data generation.

Hit Rate:
MRR:

Document -> D

D -> N1, N2, N3, N4, N5 -> Index/ Retriever

(Q1, N1)
(Q2, N1)
(Q3, N2)
(Q4, N2)
(Q5, N3)
(Q6, N3)
(Q7, N4)
(Q8, N4)
(Q9, N5)
(Q10, N5)

Q1 -> Index/ Retriever -> N2, N1, N3 -> 1 -> 1/2

Q2 -> Index/ Retriever -> N5, N4, N3 -> 0 -> 0

Q3 -> Index/ Retriever -> N1, N2, N3 -> 1 -> 1/2

Q4 -> Index/ Retriever -> N2, N3, N5 -> 1 -> 1/1

Q5 -> Index/ Retriever -> N3, N1, N4 -> 1 -> 1/1

Q6 -> Index/ Retriever -> N1, N2, N3 -> 1 -> 1/3

Q7 -> Index/ Retriever -> N4, N1, N2 -> 1 -> 1/1

Q8 -> Index/ Retriever -> N1, N3, N4 -> 1 -> 1/3

Q9 -> Index/ Retriever -> N2, N3, N4 -> 0 -> 0

Q10 -> Index/ Retriever -> N2, N5, N3 -> 1 -> 1/2

Hit Rate: 8/10 -> 80%

MRR: (0.5 + 0 + 0.5 + 1 + 1 + 0.33 + 1 + 0.33 + 0 + 0.5)/10 -> 55%

In [66]:
from llama_index.core.text_splitter import SentenceSplitter

In [67]:
from llama_index.embeddings.openai import OpenAIEmbedding

In [68]:
reader = SimpleDirectoryReader("./data/paul_graham/")
documents = reader.load_data()

# create parser and parse document into nodes
parser = SentenceSplitter(chunk_size=1024, chunk_overlap=100)
nodes = parser(documents)

In [69]:
vector_index = VectorStoreIndex(nodes, embed_model=OpenAIEmbedding(model='text-embedding-3-small'))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [70]:
# Define the retriever
retriever = vector_index.as_retriever(similarity_top_k=2)

In [71]:
retrieved_nodes = retriever.retrieve(eval_query)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [73]:
!pip install matplotlib

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.1 MB 5.6 MB/s eta 0:00:02
   ----------- ---------------------------- 2.4/8.1 MB 6.1 MB/s eta 0:00:01
   ------------------ --------------------- 3.7/8.1 MB 5.9 MB/s eta 0:00:01
   ------------------------ --------------- 5.0/8.1 MB 5.9 MB/s eta 0:00:01
   ------------------------------ --------- 6.3/8.1 MB 6.0 MB/s eta 0:00:01
   ------------------------------------ --- 7.3/8.1 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ------------------ --------------------- 1.0/2.3 MB 5.6 MB/s eta 0:00:01
   ------------------------------------- -- 2.1/2.3 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 4.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [74]:
from llama_index.core.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=2000)

**Node ID:** f380d7c1-c403-416f-9eb4-86838cc7cf17<br>**Similarity:** 0.6167315224318201<br>**Text:** What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the card reader and press a button to load the program into memory and run it. The result would ordinarily be to print something on the spectacularly loud printer.

I was puzzled by the 1401. I couldn't figure out what to do with it. And in retrospect there's not much I could have done with it. The only form of input to programs was data stored on punched cards, and I didn't have any data stored on punched cards. The only other option was to do things that didn't rely on any input, like calculate approximations of pi, but I didn't know enough math to do anything interesting of that type. So I'm not surprised I can't remember any programs I wrote, because they can't have done much. My clearest memory is of the moment I learned it was possible for programs not to terminate, when one of mine didn't. On a machine without time-sharing, this was a social as well as a technical error, as the data center manager's expression made clear.

With microcomputers, everything changed. Now you could h...<br>

**Node ID:** 76d8546d-4e69-4ae9-bf71-fb9670f4ad01<br>**Similarity:** 0.5289862943025382<br>**Text:** All I knew at the time was that I kept taking philosophy courses and they kept being boring. So I decided to switch to AI.

AI was in the air in the mid 1980s, but there were two things especially that made me want to work on it: a novel by Heinlein called The Moon is a Harsh Mistress, which featured an intelligent computer called Mike, and a PBS documentary that showed Terry Winograd using SHRDLU. I haven't tried rereading The Moon is a Harsh Mistress, so I don't know how well it has aged, but when I read it I was drawn entirely into its world. It seemed only a matter of time before we'd have Mike, and when I saw Winograd using SHRDLU, it seemed like that time would be a few years at most. All you had to do was teach SHRDLU more words.

There weren't any classes in AI at Cornell then, not even graduate classes, so I started trying to teach myself. Which meant learning Lisp, since in those days Lisp was regarded as the language of AI. The commonly used programming languages then were pretty primitive, and programmers' ideas correspondingly so. The default language at Cornell was a Pascal-like language called PL/I, and the situation was similar elsewhere. Learning Lisp expanded my concept of a program so fast that it was years before I started to have a sense of where the new limits were. This was more like it; this was what I had expected college to do. It wasn't happening in a class, like it was supposed to, but that was ok. For the next couple years I was on a roll. I knew what I was going to do.

For my undergraduate thesis, I reverse-engineered SHRDLU. My God did I love working on that program. It was a pleasing bit of code, but what made it even more exciting was my belief — hard to imagine now, but not unique in 1985 — that it was already climbing the lower slopes of intelligence.

I had gotten into a program at Cornell that didn't make you choose a major. You could take whatever classes you liked, and choose whatever you liked to put on your degree. I of c...<br>

In [75]:
qa_dataset = generate_question_context_pairs(nodes[0:2], llm=gpt4, num_questions_per_chunk=2)

  0%|          | 0/2 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 50%|█████     | 1/2 [00:01<00:01,  1.89s/it]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 2/2 [00:04<00:00,  2.33s/it]


In [76]:
qa_dataset.queries

{'200a97cc-f66d-44be-8f38-eb056c19883a': "Describe the evolution of the author's programming experiences from using the IBM 1401 to acquiring a TRS-80. How did these experiences influence their interest in programming?",
 'a792c6d8-b9ef-42c6-b8cf-fa7fba241f4e': "Discuss the author's initial perception of studying philosophy in college and how their views changed over time. What factors contributed to their decision to switch to studying AI?",
 'a91d6688-bd0b-415c-bd07-7dc6a31d42bf': "Describe the factors that influenced the author's decision to switch from studying philosophy to pursuing artificial intelligence in the mid-1980s. How did specific works and experiences contribute to this decision?",
 'a8413bde-f5e1-4c4b-a591-5ed15f7490db': "Analyze the author's perspective on the state of artificial intelligence during their first year of graduate school. What realization did they come to about AI as it was practiced at the time, and how did this impact their academic and career focus?"}

In [78]:
!pip install llama_index.llms.groq


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
from llama_index.llms.groq import Groq

In [80]:
groq = Groq(model='llama-3.2-90b-vision-preview')

In [81]:
DEFAULT_QA_GENERATE_PROMPT_TMPL = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Just provide the questions and nothing else

"""

In [83]:
qa_dataset_groq = generate_question_context_pairs(nodes[0:3], llm=groq, num_questions_per_chunk=2, 
                                                  qa_generate_prompt_tmpl=DEFAULT_QA_GENERATE_PROMPT_TMPL)

  0%|          | 0/3 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 401 Unauthorized"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 401 Unauthorized"


  0%|          | 0/3 [00:00<?, ?it/s]


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [57]:
qa_dataset_groq.queries

{'36536694-f7c2-40f1-b632-ddc8acc51487': "What was the primary input method for programs on the IBM 1401, and what limitations did this pose for the author's early programming endeavors?",
 'ac57244c-0a0a-436a-bb02-eab7fd6e1a89': "What two influences are mentioned in the text as having sparked the author's interest in pursuing Artificial Intelligence (AI) as a field of study?",
 '7062e6d4-f442-482f-81a6-5f5040758c16': 'What was the primary reason the author was drawn to learning Lisp, and how did it expand their concept of a program?',
 '0530b143-e791-4197-afa3-7345ea93cd74': 'What realization did the author come to during their first year of graduate school regarding the state of Artificial Intelligence at the time, and what implications did this have for their future plans?',
 '00ca0e9f-b720-4bde-8c22-102627c23a3f': 'What motivated the author to consider a career in art, and what realization did they have while visiting the Carnegie Institute?',
 '133d4da1-4ada-4330-b9c6-aa97d5f5b928

In [73]:
queries = qa_dataset.queries.values()
print(list(queries)[:1])

["Describe the author's initial experiences with programming on the IBM 1401. What were some of the challenges he faced and how did the advent of microcomputers change his approach to programming?"]


In [74]:
len(list(queries))

2

In [75]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [76]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Query: Describe the author's initial experiences with programming on the IBM 1401. What were some of the challenges he faced and how did the advent of microcomputers change his approach to programming?
Metrics: {'mrr': 1.0, 'hit_rate': 1.0}



In [77]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [78]:
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"retrievers": [name], "hit_rate": [hit_rate], "mrr": [mrr]}
    )

    return metric_df

In [79]:
display_results("top-2 eval", eval_results)

,retrievers,hit_rate,mrr
0,top-2 eval,1.0,1.0
